# Interactive NDVI

In [1]:
import datetime
import geemap
import ee
from astral import LocationInfo
from astral.sun import sun

ee.Initialize()
bands = ['CMI_C01', 'CMI_C02', 'CMI_C03']


def set_coordinates(left, bottom, right, top):
    return ee.Geometry.BBox(left, bottom, right, top)


def applyScaleandOffset(image):
    
    band_list = image.select('CMI_C..').bandNames()

    scales = (band_list.map(lambda name: ee.String(name).cat('_scale'))).map(lambda band: image.getNumber(band))
    offsets = (band_list.map(lambda name: ee.String(name).cat('_offset'))).map(lambda band: image.getNumber(band))

    scaled = image.select('CMI_C..').multiply(ee.Image.constant(scales)).add(ee.Image.constant(offsets))

    return image.addBands(srcImg = scaled, overwrite = True)


def get_daytime(lat, lon, day_delta = 1):

    loc = LocationInfo(latitude=lat, longitude=lon)
    s = sun(loc.observer, date=datetime.date.today() - datetime.timedelta(days= day_delta))

    if s['sunset'] <= s['sunrise']:
        s['sunset'] = s['sunset'] + datetime.timedelta(days=1)
    
    return (s['sunrise'].strftime(format="%Y-%m-%dT%H:%M:00"), s['sunset'].strftime(format="%Y-%m-%dT%H:%M:00"))


In [2]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(["CMI_C03", "CMI_C02"]).rename('NDVI')
    return image.addBands(ndvi)

def calculate_ndwi(image):

    nir = image.select("CMI_C03")
    red = image.select("CMI_C02")
    blue = image.select("CMI_C01")

    green = red.multiply(0.45).add(nir.multiply(0.1)).add(blue.multiply(0.45))
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')

    return image.addBands(ndwi)

In [9]:
def call_split_map(coords, collection_name = 'NOAA/GOES/16/MCMIPC', bands=bands):

    goes_collection = ee.ImageCollection(collection_name)

    # Set the location of the ROI. USER INPUT REQUIRED.
    roi = set_coordinates(coords['left'], coords['bottom'], coords['right'], coords['top'])

    
    # Get the image collection for the given time period and location.
    date_start, date_end = get_daytime(coords['top'], coords['left']) #UPPER LEFT CORNER (smallest day duration)
    #old_start, old_end = get_daytime(coords['top'], coords['left'], 365)
    

    period = ee.DateRange(date_start, date_end)
    #last_year = ee.DateRange(old_start, old_end)
    month_filter = ee.Filter.calendarRange(int(date_start[5:7]), 'month')

    # Calculate the indexes
    indexes = goes_collection.filterDate(period).select(bands).map(applyScaleandOffset).map(calculate_ndvi).map(calculate_ndwi).reduce(ee.Reducer.median()).clip(roi)
    #old_indexes = goes_collection.filterDate(last_year).select(bands).map(applyScaleandOffset).map(calculate_ndvi).map(calculate_ndwi).reduce(ee.Reducer.median()).clip(roi)
    history_indexes = goes_collection.filter(month_filter).select(bands).map(applyScaleandOffset).map(calculate_ndvi).map(calculate_ndwi).reduce(ee.Reducer.median()).clip(roi)

    # Split the map
    # Add the image to the map.
    ndviParams = {'bands': ['NDVI_median'],
                    'min': -1,
                    'max': 1,
                    'palette': ['blue', 'white', 'green']}



    left_layer = geemap.ee_tile_layer(history_indexes, ndviParams, 'Month NDVI')
    right_layer = geemap.ee_tile_layer(indexes, ndviParams, 'Current Year NDVI')


    colors = ndviParams['palette']
    vmin = ndviParams['min']
    vmax = ndviParams['max']


    m = geemap.Map()
    m.split_map(left_layer, right_layer)
    m.setCenter((coords['left'] + coords['right'])//2, (coords['top'] + coords['bottom'])//2, 8)
    m.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, layer_name="NDVI")

    return m

In [12]:
a, b = get_daytime(Region['top'], Region['left'])

'10'

In [10]:
Region = {
    'left' : -102.84,
    'bottom' : 37.25,
    'right' : -96.91,
    'top' : 42.27
}


call_split_map(Region)

TypeError: ComputedObject.getInfo() takes 1 positional argument but 2 were given